<a href="https://www.kaggle.com/code/jairusmartinez/cycling-energy-regression?scriptVersionId=143044467" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Problem Statement

Many endurance atheletes know what it feels like to hit the wall or to "bonk". Often times, "bonking" occurs when you burn more energy then you consume- leaving you at caloric deficit during a training ride, race, or any endurance activity. Bonking is something many people hate but experience way too often...

To mitigate bonking, you have to be on top of your caloric intake during your activity. For me specifically, when I'm cycling, I want to know how many calories I need to consume to prevent being at such a high deficit. 

Because of this, I wanted to create a cheeky little tool that can predict how many calories I burn for a given activity based on certain aspects of that activity. One approach is to use my personal cycling data with labeled energy outputs and features. I will explore my data and use it to train an ML model that can predict the energy burned in kilojoues. Based on the amount of energy burned, I can convert this number into calories which should then give me a a good estimate on how many calories I should be eating during a particular ride. 

Data was extracted using my own personal Strava Data. The python script that requests the data, transforms it, and exports it into a csv file can be found here: https://github.com/jairus-m/Strava_API/tree/main.

In addition, my full, up-to-date Strava Activity Dashboard can be found here: https://public.tableau.com/app/profile/jairusmartinez/viz/PersonalStravaActivityData/Dashboard1

This dashboard is automatically update.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
import tarfile
import urllib.request

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer

from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV
from scipy.stats import randint

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv(os.path.join(dirname, filename))
df.head()

# Exploratory Data Analysis
1. Initial filter of dataframe
    - filter by sport
    - remove categorical variables
2. Correlation
3. Distribution
4. Outliers
5. Missing Values

## Filter Data

In [ ]:
def standard_clean(df):
    '''
    Standard cleaning:
        1. filter by SportType
        2. remove unnecessary columns
        3. drop activties with missing power data

    '''
    X = df.copy()
    X = X.loc[(X['sport_type'] == 'VirtualRide') | (X['sport_type'] == 'Ride')].reset_index(drop=True)
    X = X.drop(['name', 'achievement_count', 'pr_count', 
                'average_cadence', 'time', 'time_bins', 'elapsed_time', 'date'], axis=1)
    X = X.loc[~X['average_watts'].isna()].reset_index(drop=True)
    return X


In [ ]:
df = standard_clean(df)

## Correlation

In [ ]:
df.select_dtypes(include=np.number).corr()

In [ ]:
sns.heatmap(df.select_dtypes(include=np.number).corr());

- Distance and moving time are the most correlated to kilojoules. Total elevation gain follows with average wattage being the 4th most correlated.

- Moving time and distance are highly correlated
- Average watts and normalized watts are highly correlated 
- the only negative correlation is between average speed and total elevation gain

- all these relationships make sense are intuitive to what you would expect

Note: For a baseline model, I am going to keep all of these features.  

In [ ]:
df.describe().T

In [ ]:
df.isna().sum()

Based off these two tables, we see that there is a lot of missing data from power and heartrate fields. 
    - Remove any rows with 0 power data
    - Impute rows with missing normalized power data
    - Impute missing heartrate data
In addition, looking at the max/min, we will need to address some outliers.
Looking at mean/median/std, most of the data seems normal, but we have to explore further. 

Lastly, from personal exploration/experience, there are some activties with very very bad power data due to a faulty power meter so we will have to address all of this before we create a pipeline to preprocess the data.

## Distributions and Transformations

In [ ]:
fig, ax = plt.subplots(3,3, figsize=(10,10))

flat_ax = ax.flatten()

for i, ax in enumerate(flat_ax):
    sns.histplot(data=df[df.columns[i]], ax=ax)
    
plt.show()

We see that many of the distributions are approximately normal. The two that may need some transformation are total elevation gain and kilojoules.

We will try a log transformation and square root transformation to attempt to fix these skews.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,6))

sns.histplot(np.log(df['total_elevation_gain']), ax=ax[0])
ax[0].set_title('Log Transform of Total Elevation Gain')

sns.histplot(np.sqrt(df['total_elevation_gain']), ax=ax[1])
ax[1].set_title('Square Root Transform of Total Elevation Gain')

fig.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,6))

sns.histplot(np.log(df['kilojoules']), ax=ax[0])
ax[0].set_title('Log Transform of KJs')

sns.histplot(np.sqrt(df['kilojoules']), ax=ax[1])
ax[1].set_title('Square Root Transform of KJs')

fig.show()

So far, it seems that square transforms of these two distirbutions yield the best results upon "eyeball" inspection. 

We can use a Q-Q plot or the Shapiro-Wilk test to quantitatively test out normality but we will forgo that for simplicity.

## Outliers and Dealing with Bad Data

In [ ]:
cols = ['distance', 'moving_time', 'total_elevation_gain', 'average_speed',
        'average_watts', 'weighted_average_watts', 'average_heartrate']
fig, ax = plt.subplots(7,1, figsize=(10,40))

flat_ax = ax.flatten()

for i, ax in enumerate(flat_ax):
    sns.scatterplot(x=df[cols[i]], y=df['kilojoules'], data=df, ax=ax)
    
plt.show()

Identify data that has KJ data that is too low for given distances, times, etc:

Outlier Data:
- distances greater than 20 where
    - avgerage watts is less than 80
    - sport type is outdoor rides
    - average speed is greater than 15

In [ ]:
mask = (df['distance'] >= 20 ) & (df['average_watts'] <= 80) & (df['sport_type'] == 'Ride') & (df['average_speed'] >= 15)
df.loc[mask]

In [ ]:
# drop the bad data outliers
df = df.loc[~mask]

In [ ]:
# fix century ride from broken powermeter

df.loc[df['moving_time'] > 300]


In [ ]:
df.loc[825, 'average_watts'] = 86.9*1.9
df.loc[825, 'weighted_average_watts'] = 86.9*1.9
df.loc[825, 'kilojoules'] = 1616.1 * 1.5

df.loc[825]

In [ ]:
# drop car ride data (any avg_speed over 30)
df = df.loc[df['average_speed'] < 30]

In [ ]:
cols = ['distance', 'moving_time', 'total_elevation_gain', 'average_speed',
        'average_watts', 'weighted_average_watts', 'average_heartrate']
fig, ax = plt.subplots(7,1, figsize=(10,40))

flat_ax = ax.flatten()

for i, ax in enumerate(flat_ax):
    sns.scatterplot(x=df[cols[i]], y=df['kilojoules'], data=df, ax=ax)
    
plt.show()

Now that we have eliminated most of the outlier data, we can move on to building the pipeline. However, before we do that, after getting familiar with the data/exploring and looking at correlations, I will drop the average_heartrate feature. 

Lastly, I want to see how many missing values are left after the cleaning/filtering. We will impute these using multiple methods. 

## Missing Values

In [ ]:
df.isna().sum()

We have 145 missing values for weighted_average_watts which we can impute using LinearRegression with sklearn.

# Pipeline

In [ ]:
def standard_clean(df):
    '''
    Standard cleaning:
        1. filter by SportType
        2. remove unnecessary columns
        3. drop activties with missing power data

    '''
    X = df.copy()
    
    # filter by sports type
    X = X.loc[(X['sport_type'] == 'VirtualRide') | (X['sport_type'] == 'Ride')].reset_index(drop=True)
    
    # drop unused columns
    X = X.drop(['name', 'achievement_count', 'pr_count', 
                'average_cadence', 'time', 'time_bins', 'elapsed_time', 'date', 'average_heartrate'], axis=1)
    # drop any rows with missing power data
    X = X.loc[~X['average_watts'].isna()].reset_index(drop=True)
    
    # drop outlier data based off this mask
    mask = (X['distance'] >= 20 ) & (X['average_watts'] <= 80) & (X['sport_type'] == 'Ride') & (X['average_speed'] >= 15)
    X = X.loc[~mask]
    
    return X

In [ ]:
df = standard_clean(pd.read_csv(os.path.join(dirname, filename)))
df.info()

In [ ]:
# split data

X = df.loc[:, ~df.columns.isin(['kilojoules'])]
y = np.sqrt(df.loc[:, df.columns.isin(['kilojoules'])])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24)

In [ ]:
# create the square root transform
sqrt_pipeline = Pipeline(
    steps=[
        ('impute_regression', IterativeImputer(estimator=LinearRegression())),
        ('sqrt_transform', FunctionTransformer(np.sqrt, feature_names_out='one-to-one')),
        ('strandard_scaler', StandardScaler())
    ]
)

numeric_pipeline = Pipeline(
    steps=[
        ('impute_regression', IterativeImputer(estimator=LinearRegression())),
        ('standard_scaler', StandardScaler())
    ]
)

preprocessor = ColumnTransformer(
    [
        ('nums', numeric_pipeline, ['distance', 'moving_time', 'average_speed', 'average_watts', 'weighted_average_watts']),
        ('sqrt_tranform', sqrt_pipeline, ['total_elevation_gain'])
    ], remainder='drop'
)

preprocessor

# Model Exploration and Cross Validation
1. Linear Regression
2. Random Forest Regression
3. XGBoost Regression

In [ ]:
linear_regression = Pipeline(
    steps = 
    [
        ('preprocess', preprocessor),
        ('linear_regression', LinearRegression())
    ]
)

linear_regression

In [ ]:
linear_mses = -cross_val_score(linear_regression, X_train, y_train, scoring='neg_root_mean_squared_error', cv=10)
pd.Series(linear_mses).describe().T

In [ ]:
randomForest_regression = Pipeline(
    steps = 
    [
        ('preprocess', preprocessor),
        ('random_forest', RandomForestRegressor(random_state=24))
    ]
)
randomForest_regression

In [ ]:
randomForest_mses = -cross_val_score(randomForest_regression, X_train, y_train.to_numpy().reshape(-1), scoring='neg_root_mean_squared_error', cv=10)
pd.Series(randomForest_mses).describe().T

In [ ]:
xgBoost_regression = Pipeline(
    steps = 
    [
        ('preprocess', preprocessor),
        ('xgboost', xgb.XGBRegressor())
    ]
)

xgBoost_regression

In [ ]:
xgboost_mses = -cross_val_score(xgBoost_regression, X_train, y_train, scoring='neg_root_mean_squared_error', cv=10)
pd.Series(xgboost_mses).describe().T

Based on the error and cross validation scores, XGBoost performed the best.

# Fine-tune XGBoost Model

In [ ]:
param_grid = {
    'xgboost__n_estimators': [100, 300, 600, 1000],
    'xgboost__learning_rate': [0.01, 0.1, 0.2, 0.3],
    'xgboost__max_depth': [3, 5, 7, 9]
}

grid_search = GridSearchCV(xgBoost_regression, param_grid, cv=3, scoring='neg_root_mean_squared_error')
grid_search.fit(X_train, y_train)

## Select Best Estimator and Evaluate on Test Set

In [ ]:
# this is the best model
grid_search.best_estimator_

In [ ]:
# this is the best params
grid_search.best_params_

In [ ]:
# evaluate on test data

final_mse = mean_squared_error(y_test**2, grid_search.predict(X_test)**2, squared=False)
final_mse

# Results

In [ ]:
preprocessor.fit(X_train, y_train)
for col, score in zip(preprocessor.get_feature_names_out(), grid_search.best_estimator_.named_steps['xgboost'].feature_importances_):
    print(f'{col.upper()}: {score}')

In [ ]:
final_mse

In [ ]:
raw_results = pd.DataFrame(
    {
        'Predicted Value': pd.Series(grid_search.predict(X_test)**2),
        'True Value': (y_test.reset_index(drop=True)**2)['kilojoules']
    }
).assign(Difference=lambda x: x['Predicted Value'] - x['True Value'])
raw_results

In [ ]:
# distribution of Model Residuals (Actual-Predicted) --> Since roughly normal, I am fairly confident in the performance of my model
sns.histplot(x='Difference', data=raw_results);

In [ ]:
# calculate the R-squared value for results
corr_matrix = np.corrcoef(raw_results['True Value'], raw_results['Predicted Value'])
corr = corr_matrix[0,1]
R_sq = corr**2

print(R_sq)

Our final model has an MSE of about 26. On average, the ML model is only 26 kilojoules off from the actual values. 

The top 3 most important features were:
1. distance
2. moving time
3. average watts

The 3 least important features are:
1. average speed
2. weighted average watts
3. total elevation gain

# Save the model

In [ ]:
import joblib

# this will include the preprocessing steps AND the final estimator
cycling_regression = grid_search.best_estimator_

joblib.dump(cycling_regression, 'cycling_regression.pkl')

# Limitations:
- The data is based off my cycing data and therefore, the patterns learned by the model are influenced by my physiology
    - this will not generalize to the general public as I need way more training data
- Training data was limited to about 1k activities which is not many
- My training data is biased to 1 hr rides

Future Improvements:
- Get more training data
- Test on data from other people
- Can we instead see the results when training on outside rides vs inside rides?
- What other features can we incorporate, remove, or engineer to improve our results?